In [2]:
import findspark
findspark.init()

In [6]:
import pyspark
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder \
.master('local') \
.appName('read_sensor_data') \
.config('spark.jars.packages", "com.crealytics:spark-excel_2.11:0.12.2') \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 23:51:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/05 23:51:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
# sensor 1 - xlsx

In [ ]:
# sensor 2 - csv

In [ ]:
# sensor 4 - parquet

In [ ]:
# sensor 4 diff date - pickle

In [ ]:
# sensor 5 - json